In [12]:
import numpy as np
import pandas as pd
import statsmodels.api as sm


# Data prepocessing

In [30]:
text_df = pd.read_csv('dataset/multimodal variables - Text-LIWC_Gemini-1.5-pro.csv').sort_values('video')
text_var = text_df[['self-confidence','creativity','enthusiasm','future-orientation','trustworthiness','proactive-personality']]
audio_df = pd.read_csv('dataset/multimodal variables - Audio-Sonix.csv').sort_values('video')
audio_var = audio_df[['assertiveness',	'creativity','enthusiasm',	'future-orientation',	'optimism',	'proactive-personality','self-confidence']]
video_df = pd.read_csv('dataset/multimodal variables - Video-Gemini-1.5-Pro.csv').sort_values('video')
video_var = video_df[['self-confidence',	'creativity','enthusiasm',	'trustworthiness',	'optimism',	'proactive-personality','future-orientation']]
startup_df = pd.read_csv('dataset/Startup_New_backUP.xlsx - Sheet1.csv')
original_var = startup_df[['num_founders','Priorsharedexperience','Industryexperience','educationalqualification']]
original_var = original_var.apply(pd.to_numeric, errors='coerce')
y_var = startup_df[['Fundraised','Backers']]
y_var['Fundraised'] = y_var['Fundraised'].copy().apply(lambda x: x.replace('$','').replace(',','')).astype(float)
y_var['Backers'] = y_var['Backers'].copy().apply(lambda x: x.replace('$','').replace(',','')).astype(float)

/var/folders/t9/x5knz49s7qj_dry4y1r799x00000gn/T/ipykernel_90498/2183331632.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y_var['Fundraised'] = y_var['Fundraised'].copy().apply(lambda x: x.replace('$','').replace(',','')).astype(float)
/var/folders/t9/x5knz49s7qj_dry4y1r799x00000gn/T/ipykernel_90498/2183331632.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y_var['Backers'] = y_var['Backers'].copy().apply(lambda x: x.replace('$','').replace(',','')).astype(float)


In [32]:
for var_df in [original_var, text_var, audio_var, video_var]:
    #impute missing values with mean
    
    var_df.fillna(var_df.mean(), inplace=True)

y_fundraised = y_var['Fundraised']
y_backers = y_var['Backers']

/var/folders/t9/x5knz49s7qj_dry4y1r799x00000gn/T/ipykernel_90498/505711076.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  var_df.fillna(var_df.mean(), inplace=True)


# Model 0(Benchmark)

In [33]:
original_var

,num_founders,Priorsharedexperience,Industryexperience,educationalqualification
0,3.0,1.000000,0.000000,0.000000
1,1.0,0.000000,1.000000,1.000000
2,1.0,0.000000,0.000000,1.000000
3,2.0,0.000000,0.000000,0.000000
4,1.0,0.099099,0.216216,0.490741
...,...,...,...,...
112,1.0,0.000000,0.000000,1.000000
113,2.0,0.000000,0.000000,0.000000
114,2.0,0.000000,1.000000,1.000000
115,1.0,0.000000,1.000000,1.000000


In [34]:
# regression with basic variables
# FUNDRAISED
X = pd.DataFrame(original_var)
X = sm.add_constant(X)

model = sm.OLS(y_fundraised, X).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             Fundraised   R-squared:                       0.062
Model:                            OLS   Adj. R-squared:                  0.028
Method:                 Least Squares   F-statistic:                     1.851
Date:                Thu, 23 Jan 2025   Prob (F-statistic):              0.124
Time:                        16:28:29   Log-Likelihood:                -1342.1
No. Observations:                 117   AIC:                             2694.
Df Residuals:                     112   BIC:                             2708.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
============================================================================================
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
const                     1.315e+04   4322.919      3.041      0.003    4580.622    2.17e+04
num_founders              1447.3241   2444.725      0.592      0.555   -3396.586    6291.234
Priorsharedexperience    -1762.6033   8128.045     -0.217      0.829   -1.79e+04    1.43e+04
Industryexperience        5134.2715   5562.623      0.923      0.358   -5887.353    1.62e+04
educationalqualification  5291.3242   2207.690      2.397      0.018     917.069    9665.580
==============================================================================
Omnibus:                      123.816   Durbin-Watson:                   1.819
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1982.151
Skew:                           3.692   Prob(JB):                         0.00
Kurtosis:                      21.764   Cond. No.                         7.78
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [35]:
model = sm.OLS(y_backers, X).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                Backers   R-squared:                       0.064
Model:                            OLS   Adj. R-squared:                  0.030
Method:                 Least Squares   F-statistic:                     1.906
Date:                Thu, 23 Jan 2025   Prob (F-statistic):              0.114
Time:                        16:28:57   Log-Likelihood:                -840.73
No. Observations:                 117   AIC:                             1691.
Df Residuals:                     112   BIC:                             1705.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
============================================================================================
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
const                      193.6718     59.529      3.253      0.002      75.722     311.621
num_founders                34.0676     33.665      1.012      0.314     -32.636     100.771
Priorsharedexperience      -24.9210    111.928     -0.223      0.824    -246.693     196.851
Industryexperience          87.9539     76.601      1.148      0.253     -63.821     239.729
educationalqualification    65.7736     30.401      2.164      0.033       5.537     126.010
==============================================================================
Omnibus:                       57.821   Durbin-Watson:                   1.637
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              154.721
Skew:                           1.956   Prob(JB):                     2.53e-34
Kurtosis:                       7.054   Cond. No.                         7.78
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

# Model 1(Text Only)

In [36]:
# regression with text variables without original variables on fundraised

X = pd.concat([text_var], axis=1)
X = sm.add_constant(X)
y_fundraised = y_var['Fundraised']

model = sm.OLS(y_fundraised, X).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             Fundraised   R-squared:                       0.076
Model:                            OLS   Adj. R-squared:                  0.026
Method:                 Least Squares   F-statistic:                     1.513
Date:                Thu, 23 Jan 2025   Prob (F-statistic):              0.181
Time:                        16:29:17   Log-Likelihood:                -1341.2
No. Observations:                 117   AIC:                             2696.
Df Residuals:                     110   BIC:                             2716.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                  1.924e+04   2.29e+04      0.840      0.403   -2.62e+04    6.47e+04
self-confidence           6.7892     90.055      0.075      0.940    -171.679     185.258
creativity             3874.0730   1607.972      2.409      0.018     687.451    7060.695
enthusiasm             -233.1434    469.810     -0.496      0.621   -1164.196     697.909
future-orientation    -3622.6905   3048.007     -1.189      0.237   -9663.124    2417.743
trustworthiness       -1577.5434   1213.231     -1.300      0.196   -3981.883     826.796
proactive-personality -1245.3370   2434.990     -0.511      0.610   -6070.916    3580.242
==============================================================================
Omnibus:                      122.384   Durbin-Watson:                   1.772
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1979.058
Skew:                           3.616   Prob(JB):                         0.00
Kurtosis:                      21.806   Cond. No.                         862.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [38]:
# regression with text variables without original variables on backers
y_backers = y_var['Backers']
model = sm.OLS(y_backers, X).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                Backers   R-squared:                       0.054
Model:                            OLS   Adj. R-squared:                  0.003
Method:                 Least Squares   F-statistic:                     1.055
Date:                Thu, 23 Jan 2025   Prob (F-statistic):              0.394
Time:                        16:29:40   Log-Likelihood:                -841.31
No. Observations:                 117   AIC:                             1697.
Df Residuals:                     110   BIC:                             1716.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                   186.4009    319.509      0.583      0.561    -446.790     819.592
self-confidence           0.3355      1.256      0.267      0.790      -2.153       2.824
creativity               32.1740     22.423      1.435      0.154     -12.264      76.612
enthusiasm               -4.9199      6.552     -0.751      0.454     -17.904       8.064
future-orientation      -72.8670     42.505     -1.714      0.089    -157.102      11.368
trustworthiness         -10.7113     16.919     -0.633      0.528     -44.240      22.818
proactive-personality     5.2913     33.956      0.156      0.876     -62.002      72.585
==============================================================================
Omnibus:                       47.586   Durbin-Watson:                   1.524
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               96.922
Skew:                           1.750   Prob(JB):                     8.99e-22
Kurtosis:                       5.761   Cond. No.                         862.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [39]:
# regression with text variables and original variables on fundraised

X = pd.concat([text_var,original_var], axis=1)
X = sm.add_constant(X)
y_fundraised = y_var['Fundraised']

model = sm.OLS(y_fundraised, X).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             Fundraised   R-squared:                       0.140
Model:                            OLS   Adj. R-squared:                  0.059
Method:                 Least Squares   F-statistic:                     1.723
Date:                Thu, 23 Jan 2025   Prob (F-statistic):             0.0849
Time:                        16:30:07   Log-Likelihood:                -1337.0
No. Observations:                 117   AIC:                             2696.
Df Residuals:                     106   BIC:                             2726.
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
============================================================================================
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
const                     1.717e+04   2.31e+04      0.743      0.459   -2.86e+04     6.3e+04
self-confidence             -8.8379     92.094     -0.096      0.924    -191.423     173.747
creativity                3709.0593   1610.885      2.302      0.023     515.323    6902.796
enthusiasm                -459.2056    472.952     -0.971      0.334   -1396.879     478.468
future-orientation       -3489.9095   3032.757     -1.151      0.252   -9502.646    2522.827
trustworthiness          -1757.5261   1206.316     -1.457      0.148   -4149.166     634.113
proactive-personality    -1023.3961   2458.768     -0.416      0.678   -5898.144    3851.352
num_founders              1141.7928   2498.570      0.457      0.649   -3811.866    6095.451
Priorsharedexperience    -4410.4175   8144.559     -0.542      0.589   -2.06e+04    1.17e+04
Industryexperience        5538.6892   5584.141      0.992      0.324   -5532.413    1.66e+04
educationalqualification  5445.0595   2268.791      2.400      0.018     946.961    9943.158
==============================================================================
Omnibus:                      126.686   Durbin-Watson:                   1.776
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             2349.186
Skew:                           3.743   Prob(JB):                         0.00
Kurtosis:                      23.636   Cond. No.                         884.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [40]:
# regression with text variables with original variables on backers
y_backers = y_var['Backers']
model = sm.OLS(y_backers, X).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                Backers   R-squared:                       0.120
Model:                            OLS   Adj. R-squared:                  0.037
Method:                 Least Squares   F-statistic:                     1.440
Date:                Thu, 23 Jan 2025   Prob (F-statistic):              0.173
Time:                        16:30:26   Log-Likelihood:                -837.13
No. Observations:                 117   AIC:                             1696.
Df Residuals:                     106   BIC:                             1727.
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
============================================================================================
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
const                      118.8443    322.038      0.369      0.713    -519.628     757.316
self-confidence              0.0073      1.284      0.006      0.995      -2.539       2.553
creativity                  29.4547     22.463      1.311      0.193     -15.080      73.989
enthusiasm                  -7.7442      6.595     -1.174      0.243     -20.819       5.331
future-orientation         -70.4651     42.290     -1.666      0.099    -154.308      13.378
trustworthiness            -12.6040     16.821     -0.749      0.455     -45.954      20.746
proactive-personality       10.3382     34.286      0.302      0.764     -57.637      78.313
num_founders                33.1775     34.841      0.952      0.343     -35.898     102.253
Priorsharedexperience      -68.4842    113.570     -0.603      0.548    -293.648     156.679
Industryexperience          89.1329     77.867      1.145      0.255     -65.246     243.511
educationalqualification    68.9404     31.637      2.179      0.032       6.218     131.663
==============================================================================
Omnibus:                       50.346   Durbin-Watson:                   1.621
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              114.330
Skew:                           1.771   Prob(JB):                     1.49e-25
Kurtosis:                       6.304   Cond. No.                         884.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

# Model2 (Text+Audio)

In [41]:
# regression with text + audio variables without original variables on backers

X = pd.concat([text_var[['creativity','future-orientation']], audio_var[['self-confidence','enthusiasm']],text_var[['trustworthiness','proactive-personality']]], axis=1)
X = sm.add_constant(X)

model = sm.OLS(y_fundraised, X).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             Fundraised   R-squared:                       0.108
Model:                            OLS   Adj. R-squared:                  0.059
Method:                 Least Squares   F-statistic:                     2.209
Date:                Thu, 23 Jan 2025   Prob (F-statistic):             0.0473
Time:                        16:31:32   Log-Likelihood:                -1339.2
No. Observations:                 117   AIC:                             2692.
Df Residuals:                     110   BIC:                             2712.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                  1.651e+04   2.83e+04      0.583      0.561   -3.96e+04    7.27e+04
creativity             2394.5678   1771.258      1.352      0.179   -1115.650    5904.785
future-orientation    -3585.8829   3005.635     -1.193      0.235   -9542.345    2370.580
self-confidence        5640.8536   2813.671      2.005      0.047      64.818    1.12e+04
enthusiasm            -4032.2809   2929.106     -1.377      0.171   -9837.082    1772.520
trustworthiness       -1526.3868   1186.422     -1.287      0.201   -3877.597     824.823
proactive-personality -1222.2715   2389.687     -0.511      0.610   -5958.070    3513.527
==============================================================================
Omnibus:                      118.908   Durbin-Watson:                   1.707
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1879.417
Skew:                           3.464   Prob(JB):                         0.00
Kurtosis:                      21.372   Cond. No.                         230.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [42]:
# regression with text + audio variables without original variables on backers
model = sm.OLS(y_backers, X).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                Backers   R-squared:                       0.053
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     1.021
Date:                Thu, 23 Jan 2025   Prob (F-statistic):              0.416
Time:                        16:31:49   Log-Likelihood:                -841.42
No. Observations:                 117   AIC:                             1697.
Df Residuals:                     110   BIC:                             1716.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                   239.8386    402.378      0.596      0.552    -557.580    1037.257
creativity               25.7891     25.152      1.025      0.307     -24.057      75.635
future-orientation      -68.9854     42.681     -1.616      0.109    -153.569      15.598
self-confidence          21.7230     39.955      0.544      0.588     -57.459     100.905
enthusiasm              -23.8297     41.594     -0.573      0.568    -106.260      58.600
trustworthiness         -11.7504     16.848     -0.697      0.487     -45.138      21.638
proactive-personality     5.5590     33.934      0.164      0.870     -61.691      72.809
==============================================================================
Omnibus:                       49.626   Durbin-Watson:                   1.517
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              106.519
Skew:                           1.791   Prob(JB):                     7.41e-24
Kurtosis:                       6.004   Cond. No.                         230.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [43]:
# regression with text + audio variables and original variables on fundraised

X = pd.concat([text_var[['creativity','future-orientation']], audio_var[['self-confidence','enthusiasm']],text_var[['trustworthiness','proactive-personality']], original_var], axis=1)
X = sm.add_constant(X)

model = sm.OLS(y_fundraised, X).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             Fundraised   R-squared:                       0.154
Model:                            OLS   Adj. R-squared:                  0.075
Method:                 Least Squares   F-statistic:                     1.935
Date:                Thu, 23 Jan 2025   Prob (F-statistic):             0.0482
Time:                        16:32:17   Log-Likelihood:                -1336.0
No. Observations:                 117   AIC:                             2694.
Df Residuals:                     106   BIC:                             2724.
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
============================================================================================
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
const                     1.545e+04   2.85e+04      0.542      0.589    -4.1e+04    7.19e+04
creativity                2452.5519   1787.125      1.372      0.173   -1090.597    5995.701
future-orientation       -3281.3804   3015.259     -1.088      0.279   -9259.425    2696.665
self-confidence           4701.2842   2854.832      1.647      0.103    -958.699    1.04e+04
enthusiasm               -3566.4442   2965.301     -1.203      0.232   -9445.443    2312.554
trustworthiness          -1781.4609   1192.492     -1.494      0.138   -4145.691     582.770
proactive-personality    -1074.0696   2417.938     -0.444      0.658   -5867.868    3719.729
num_founders               842.4417   2451.653      0.344      0.732   -4018.199    5703.082
Priorsharedexperience    -4335.7482   8114.685     -0.534      0.594   -2.04e+04    1.18e+04
Industryexperience        4115.8319   5603.471      0.735      0.464   -6993.594    1.52e+04
educationalqualification  4732.4773   2203.602      2.148      0.034     363.622    9101.333
==============================================================================
Omnibus:                      122.297   Durbin-Watson:                   1.720
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             2116.975
Skew:                           3.572   Prob(JB):                         0.00
Kurtosis:                      22.576   Cond. No.                         234.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [44]:
# regression with text + audio variables and original variables on backers
model = sm.OLS(y_backers, X).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                Backers   R-squared:                       0.110
Model:                            OLS   Adj. R-squared:                  0.025
Method:                 Least Squares   F-statistic:                     1.303
Date:                Thu, 23 Jan 2025   Prob (F-statistic):              0.238
Time:                        16:32:33   Log-Likelihood:                -837.80
No. Observations:                 117   AIC:                             1698.
Df Residuals:                     106   BIC:                             1728.
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
============================================================================================
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
const                      188.2093    402.990      0.467      0.641    -610.757     987.175
creativity                  28.0453     25.277      1.110      0.270     -22.070      78.160
future-orientation         -63.1121     42.648     -1.480      0.142    -147.666      21.442
self-confidence              3.5676     40.379      0.088      0.930     -76.488      83.623
enthusiasm                 -16.4117     41.942     -0.391      0.696     -99.565      66.742
trustworthiness            -14.5571     16.867     -0.863      0.390     -47.997      18.883
proactive-personality        9.0926     34.200      0.266      0.791     -58.712      76.897
num_founders                33.3443     34.677      0.962      0.338     -35.405     102.094
Priorsharedexperience      -60.7340    114.775     -0.529      0.598    -288.287     166.819
Industryexperience          89.4487     79.256      1.129      0.262     -67.685     246.582
educationalqualification    61.4911     31.168      1.973      0.051      -0.303     123.285
==============================================================================
Omnibus:                       53.274   Durbin-Watson:                   1.618
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              128.918
Skew:                           1.843   Prob(JB):                     1.01e-28
Kurtosis:                       6.586   Cond. No.                         234.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

# Model3 (Text+Audio+Video)

In [45]:
# regression with text + audio+video variables without original variables on fundraised

X = pd.concat([video_var[['creativity','future-orientation']], audio_var[['self-confidence','enthusiasm']],text_var[['proactive-personality','trustworthiness']]], axis=1)
X = sm.add_constant(X)

model = sm.OLS(y_fundraised, X).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             Fundraised   R-squared:                       0.087
Model:                            OLS   Adj. R-squared:                  0.038
Method:                 Least Squares   F-statistic:                     1.756
Date:                Thu, 23 Jan 2025   Prob (F-statistic):              0.115
Time:                        16:33:02   Log-Likelihood:                -1340.5
No. Observations:                 117   AIC:                             2695.
Df Residuals:                     110   BIC:                             2714.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                 -6042.2403   3.37e+04     -0.179      0.858   -7.29e+04    6.08e+04
creativity             1734.4685   1835.204      0.945      0.347   -1902.476    5371.413
future-orientation      941.3499   1784.619      0.527      0.599   -2595.346    4478.045
self-confidence        7241.8159   2603.754      2.781      0.006    2081.788    1.24e+04
enthusiasm            -4896.7297   3008.178     -1.628      0.106   -1.09e+04    1064.773
proactive-personality  -525.5407   2307.042     -0.228      0.820   -5097.557    4046.476
trustworthiness       -1556.9181   1189.857     -1.308      0.193   -3914.936     801.099
==============================================================================
Omnibus:                      114.367   Durbin-Watson:                   1.726
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1606.237
Skew:                           3.322   Prob(JB):                         0.00
Kurtosis:                      19.892   Cond. No.                         313.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [ ]:
# regression with text + audio+video variables without original variables on backers
model = sm.OLS(y_backers, X).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                Backers   R-squared:                       0.029
Model:                            OLS   Adj. R-squared:                 -0.024
Method:                 Least Squares   F-statistic:                    0.5404
Date:                Thu, 23 Jan 2025   Prob (F-statistic):              0.776
Time:                        16:33:13   Log-Likelihood:                -842.89
No. Observations:                 117   AIC:                             1700.
Df Residuals:                     110   BIC:                             1719.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                    33.5450    479.496      0.070      0.944    -916.704     983.794
creativity               28.4346     26.097      1.090      0.278     -23.284      80.153
future-orientation       -2.6930     25.378     -0.106      0.916     -52.986      47.600
self-confidence          35.9018     37.026      0.970      0.334     -37.476     109.279
enthusiasm              -30.9510     42.777     -0.724      0.471    -115.726      53.824
proactive-personality     9.9954     32.807      0.305      0.761     -55.020      75.011
trustworthiness         -13.2842     16.920     -0.785      0.434     -46.816      20.248
==============================================================================
Omnibus:                       50.801   Durbin-Watson:                   1.509
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              111.182
Skew:                           1.825   Prob(JB):                     7.20e-25
Kurtosis:                       6.080   Cond. No.                         313.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [47]:
# regression with text + audio+video variables and original variables on fundraised

X = pd.concat([video_var[['creativity','future-orientation']], audio_var[['self-confidence','enthusiasm']],text_var[['proactive-personality','trustworthiness']], original_var], axis=1)
X = sm.add_constant(X)

model = sm.OLS(y_fundraised, X).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             Fundraised   R-squared:                       0.135
Model:                            OLS   Adj. R-squared:                  0.053
Method:                 Least Squares   F-statistic:                     1.656
Date:                Thu, 23 Jan 2025   Prob (F-statistic):              0.101
Time:                        16:33:49   Log-Likelihood:                -1337.4
No. Observations:                 117   AIC:                             2697.
Df Residuals:                     106   BIC:                             2727.
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
============================================================================================
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
const                    -6307.3485   3.39e+04     -0.186      0.853   -7.36e+04     6.1e+04
creativity                1631.9541   1861.223      0.877      0.383   -2058.102    5322.010
future-orientation         736.7201   1797.958      0.410      0.683   -2827.906    4301.346
self-confidence           6298.0149   2649.365      2.377      0.019    1045.391    1.16e+04
enthusiasm               -4261.3773   3067.870     -1.389      0.168   -1.03e+04    1820.974
proactive-personality     -217.0023   2321.264     -0.093      0.926   -4819.134    4385.129
trustworthiness          -1874.5006   1200.029     -1.562      0.121   -4253.675     504.674
num_founders              1075.0595   2473.428      0.435      0.665   -3828.753    5978.872
Priorsharedexperience    -1964.4752   8313.520     -0.236      0.814   -1.84e+04    1.45e+04
Industryexperience        1815.0855   5678.423      0.320      0.750   -9442.941    1.31e+04
educationalqualification  5171.6613   2215.043      2.335      0.021     780.124    9563.199
==============================================================================
Omnibus:                      117.754   Durbin-Watson:                   1.755
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1798.652
Skew:                           3.430   Prob(JB):                         0.00
Kurtosis:                      20.941   Cond. No.                         318.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [48]:
# regression with text + audio+video variables and original variables on backers

model = sm.OLS(y_backers, X).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                Backers   R-squared:                       0.089
Model:                            OLS   Adj. R-squared:                  0.003
Method:                 Least Squares   F-statistic:                     1.032
Date:                Thu, 23 Jan 2025   Prob (F-statistic):              0.422
Time:                        16:34:06   Log-Likelihood:                -839.15
No. Observations:                 117   AIC:                             1700.
Df Residuals:                     106   BIC:                             1731.
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
============================================================================================
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
const                        9.2633    480.140      0.019      0.985    -942.661     961.187
creativity                  26.8410     26.332      1.019      0.310     -25.366      79.048
future-orientation          -8.8412     25.437     -0.348      0.729     -59.273      41.591
self-confidence             16.9326     37.483      0.452      0.652     -57.381      91.246
enthusiasm                 -18.8316     43.404     -0.434      0.665    -104.884      67.221
proactive-personality       16.3959     32.841      0.499      0.619     -48.715      81.507
trustworthiness            -16.5768     16.978     -0.976      0.331     -50.237      17.084
num_founders                37.9097     34.994      1.083      0.281     -31.469     107.289
Priorsharedexperience      -16.0254    117.619     -0.136      0.892    -249.217     217.166
Industryexperience          67.0843     80.338      0.835      0.406     -92.194     226.362
educationalqualification    68.2937     31.338      2.179      0.032       6.162     130.425
==============================================================================
Omnibus:                       54.362   Durbin-Watson:                   1.619
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              132.009
Skew:                           1.887   Prob(JB):                     2.16e-29
Kurtosis:                       6.582   Cond. No.                         318.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""